Objective:  Predict survivors of Titanic sinking

Methodolgy:  Use several ML models from Udemy class to improve accuracy of survivor predictions on test data and submit best  
             Replaced categorical columns, male/female as binary, and dummy variables for Pclass, Embarked, and Cabin letter  
             Additional test and train data with Age and Fare scaled for models that perform better with scaled  

Discussion:  Titanic training Random Forest score was 77.5% using only Pclass, Sex, SibSp and Parch   
             Naive Bays unscaled was 75.8%  
             Decision Tree unscaled 78.1%  
             Linear Regression either 80.1%  
             KNN scaled 81.104%  
             Random Forest unscaled 81.108% 
             SVC scaled 82.2%  
             XGBoost either 82.6%  
             CatBoost either 82.7%
             
             

In [88]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [91]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
dfs = [train_data, test_data]
combined_data = pd.concat(dfs)

In [92]:
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
dtypes: float64(3), int64(4), object(5)
memory usage: 132.9+ KB


Age is 80% populated, while Cabin is only 23% populated.  Embarked is only missing two records which I will exclude.

In [54]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 889 entries, 0 to 890
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  889 non-null    int64  
 1   Survived     889 non-null    int64  
 2   Pclass       889 non-null    int64  
 3   Name         889 non-null    object 
 4   Sex          889 non-null    object 
 5   Age          889 non-null    float64
 6   SibSp        889 non-null    int64  
 7   Parch        889 non-null    int64  
 8   Ticket       889 non-null    object 
 9   Fare         889 non-null    float64
 10  Cabin        202 non-null    object 
 11  Embarked     889 non-null    object 
 12  cabin_ltr    889 non-null    object 
dtypes: float64(2), int64(5), object(6)
memory usage: 129.5+ KB


No null Embarked on test data

In [93]:
train_data.dropna(subset=['Embarked'],inplace = True)

In [96]:
combined_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [97]:
combined_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,1309.000000,891.000000,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000
mean,655.000000,0.383838,2.294882,29.881138,0.498854,0.385027,33.295479
std,378.020061,0.486592,0.837836,14.413493,1.041658,0.865560,51.758668
min,1.000000,0.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,328.000000,0.000000,2.000000,21.000000,0.000000,0.000000,7.895800
50%,655.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,982.000000,1.000000,3.000000,39.000000,1.000000,0.000000,31.275000
max,1309.000000,1.000000,3.000000,80.000000,8.000000,9.000000,512.329200


In [98]:
combined_data.Fare[combined_data.Fare ==0].count()

17

Fare has 17 zeros, replace with non-zero mean

In [99]:
mean_fare = combined_data[combined_data['Fare'] != 0]['Fare'].mean()
train_data['Fare'] = train_data['Fare'].replace(0, mean_fare)
train_data['Fare'] = train_data['Fare'].fillna(mean_fare)
test_data['Fare'] = test_data['Fare'].replace(0, mean_fare)
test_data['Fare'] = test_data['Fare'].fillna(mean_fare)

Update missing ages with average by gender

In [100]:
mean_age_male = combined_data[combined_data['Sex'] == 'male']['Age'].mean()
mean_age_female = combined_data[combined_data['Sex'] == 'female']['Age'].mean()
print(mean_age_male)
print(mean_age_female)

30.58522796352584
28.68708762886598


In [101]:
train_data.loc[(train_data['Sex'] == 'male') & (train_data['Age'].isnull()),'Age'] = mean_age_male
test_data.loc[(test_data['Sex'] == 'male') & (test_data['Age'].isnull()),'Age'] = mean_age_male
train_data.loc[(train_data['Sex'] == 'female') & (train_data['Age'].isnull()),'Age'] = mean_age_female
test_data.loc[(test_data['Sex'] == 'female') & (test_data['Age'].isnull()),'Age'] = mean_age_female

In [ ]:
Cabin has starting letters that may group people into similar locations on the ship

In [102]:
train_data['cabin_ltr'] = train_data.Cabin.apply(lambda x: str(x)[0])
test_data['cabin_ltr'] = test_data.Cabin.apply(lambda x: str(x)[0])

Convert male and female to binary

In [103]:
train_data['Sex'] = train_data['Sex'].map({'male':0, 'female':1})
test_data['Sex'] = test_data['Sex'].map({'male':0, 'female':1})

Replace categorical with dummies


In [104]:
cat_cols = ["Pclass", "Sex", "Embarked", "cabin_ltr"]
train_encoded = pd.get_dummies(data = train_data, columns = cat_cols)
test_encoded = pd.get_dummies(data = test_data, columns = cat_cols)


Drop unneeded columns

In [105]:
train_encoded.drop(['PassengerId', 'Ticket', 'Name', 'Cabin'],axis = 1, inplace = True)
test_encoded.drop(['PassengerId', 'Ticket', 'Name', 'Cabin'],axis = 1, inplace = True)

In [107]:
y_train = train_encoded['Survived']
X_train = train_encoded.drop(['Survived'], axis = 1)
X_test = test_encoded.copy()

Scale the Age and Fare

In [108]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()
X_train_scaled[['Age', 'Fare']] = sc.fit_transform(X_train_scaled[['Age', 'Fare']])
X_test_scaled[['Age', 'Fare']] = sc.fit_transform(X_test_scaled[['Age', 'Fare']])

In [130]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score

In [110]:

clf = GaussianNB()
cv = cross_val_score(clf, X_train, y_train, cv = 5)
print(cv.mean())

0.7581920903954803


clf = GaussianNB()
cv = cross_val_score(clf, X_train_scaled, y_train, cv = 5)
print(cv.mean())

In [117]:
clf = LogisticRegression(max_iter = 1000)
cv = cross_val_score(clf, X_train_scaled, y_train, cv = 5)
print(cv.mean())

0.8009268075922046


In [122]:
clf = tree.DecisionTreeClassifier()
cv = cross_val_score(clf, X_train, y_train, cv = 5)
print(cv.mean())

0.7806893924966674


In [123]:
clf = KNeighborsClassifier()
cv = cross_val_score(clf, X_train_scaled, y_train, cv = 5)
print(cv.mean())

0.8110391671427666


In [125]:
clf = RandomForestClassifier(random_state = 7)
cv = cross_val_score(clf, X_train, y_train, cv = 5)
print(cv.mean())

0.8110772551260077


In [127]:
clf = SVC(probability = True)
cv = cross_val_score(clf, X_train_scaled, y_train, cv = 5)
print(cv.mean())

0.8223132101821875


In [132]:
clf = XGBClassifier(random_state = 7)
cv = cross_val_score(clf, X_train_scaled, y_train, cv = 5)
print(cv.mean())

0.8256649527074208


In [133]:
clf = CatBoostClassifier()
cv = cross_val_score(clf, X_train_scaled, y_train, cv = 5)
print(cv.mean())

Learning rate set to 0.008906
0:	learn: 0.6876257	total: 54.4ms	remaining: 54.3s
1:	learn: 0.6826955	total: 56.7ms	remaining: 28.3s
2:	learn: 0.6777138	total: 58.6ms	remaining: 19.5s
3:	learn: 0.6724264	total: 60.2ms	remaining: 15s
4:	learn: 0.6666867	total: 61.8ms	remaining: 12.3s
5:	learn: 0.6613809	total: 63.5ms	remaining: 10.5s
6:	learn: 0.6565170	total: 65.1ms	remaining: 9.23s
7:	learn: 0.6517487	total: 66.7ms	remaining: 8.27s
8:	learn: 0.6465653	total: 68.3ms	remaining: 7.51s
9:	learn: 0.6416864	total: 69.9ms	remaining: 6.92s
10:	learn: 0.6367716	total: 71.5ms	remaining: 6.43s
11:	learn: 0.6320533	total: 73.1ms	remaining: 6.01s
12:	learn: 0.6273078	total: 75.1ms	remaining: 5.7s
13:	learn: 0.6229345	total: 76.6ms	remaining: 5.4s
14:	learn: 0.6182738	total: 78.4ms	remaining: 5.14s
15:	learn: 0.6133244	total: 80ms	remaining: 4.92s
16:	learn: 0.6094410	total: 81.7ms	remaining: 4.72s
17:	learn: 0.6047722	total: 83.3ms	remaining: 4.55s
18:	learn: 0.6006199	total: 85.1ms	remaining: 4.39